# References:
- https://www.npr.org/sections/thesalt/2017/11/30/565769194/i-was-poisoned-can-crowdsourcing-food-illnesses-help-stop-outbreaks <br>
- https://clark.com/shopping-retail/food-restaurants/i-was-poisoned-website-restaurant-reports/ <br>
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5517822/ <br>

### Software Extensions
- Current implementation uses Beautiful Soup which should work until IWP.com changes their portal format
- Scrapy framework was identified late. In future may choose to build web spider https://scrapy.org/

In [1]:
import re
import csv
import time
import json
import uuid
import glob
import copy
import string
import base64
import string
import hashlib
import subprocess
import collections
from collections import OrderedDict

import urllib2
import requests
import usaddress
import numpy as np
import pprint as pp
import pandas as pd

from bs4 import BeautifulSoup
from HTMLParser import HTMLParser
from collections import defaultdict
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_colwidth', -1)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def stripspaces(text):
    return re.sub(' +', ' ', text, flags=re.UNICODE).rstrip()

def merged_states(row):
    if pd.isnull(row[0]):
        return np.nan
    if len(row[0]) == 2:
        return row[0]
    else:
        return row[1]

# Establish Hive table columns and schema formats

In [2]:
my_cols = [u'uid', u'source', 'IWP_brand', 'Brand_prefix', u'date_time', u"Report Type",
           u'Symptoms', u'Diagnosis', u'Doctor Visit', u'Details', u'AddressNumber', u'BuildingName',
           u'CountryName', u'Location', u'PlaceName', u'StateName', u'StreetName', u'StreetNamePostDirectional',
           u'StreetNamePostType', u'StreetNamePreDirectional', u'ZipCode']

my_index = 'uid'

my_formats = {"uid": unicode, "source": unicode, "IWP_brand":unicode, 'Brand_prefix': unicode,
    "date_time": pd.to_datetime, "Report Type": unicode, "Symptoms": unicode, "Diagnosis":unicode,
    "Doctor Visit":unicode, "Details":unicode, "AddressNumber":str, "BuildingName":unicode,
    "CountryName":unicode, "Location":unicode, "PlaceName":unicode, "StateName":unicode,
    "StreetName":unicode, "StreetNamePostDirectional":unicode, "StreetNamePostType":unicode,
    "StreetNamePreDirectional":unicode, "ZipCode":unicode}

data_path = "/Users/loubellaire/Documents/_SMG/Epic_Planning/DataScience/Text Processing/Alerts/food_illness/IWP_tableformat/"

# Load reference data
- Geographic state name standards and (FIPS) values
- For future -- spell correct the brand names using SMG dictionaries via GitHub (norm_source pull request)

In [3]:
df_fips = pd.read_excel(data_path + "US_States_FIPS_codes.xlsx")
df_fips = df_fips.rename(columns = lambda x: x.strip())

```
norm_source = "https://raw.github.com/ServiceManagementGroup/datascience-dictionaries/blob/master/entities/Brands/normalization.dat"
requests.get(norm_source, auth=('rlbellaire', '1234qwer')) 
```

# Map IWasPoisoned.com Brands
- mapping Iwaspoisoned.com (IWP) brands to SMG_KC brand prefix or (if not client) SMini parent brand.
- Do some normalization of IWP brand names to get more accurate counts/brand

In [4]:
#format IWP: SMG (VV code or SM Parent brand)

brand_prefix_dict ={"IHOP": "DINE_IHOP", u"7-Eleven": u"7-Eleven",
    u'99 Cents Only Stores': u"99 Cents Only Store", "AMC": "AMC", u'ALDI': u"Aldi", 
    u'Applebee\u2019s': "DINE_APP", u"Arby's" : u"Arby's Restaurant Group, Inc.",
    u"Auntie Anne's Pretzels": "Auntie Anne's", u'Bahama Breeze' : "DAR", 
    u"Chipotle Mexican Grill": u"Chipotle", u"McDonald's": u"MCD", u'Taco Bell': u"TAC",
    u"Wendy's": "WEND", u'Burger King' : u"BK", u'Chick-fil-A': u"CFA",
    u'SUBWAY\xaeRestaurants': "Doctor's Associates, Inc.", u'Starbucks': "STAR",
    u'Panera Bread': u"Panera", u'Panda Express': "Panda Express",
    u'Buffalo Wild Wings': "BUFA", u'Pizza Hut': "PHI", u'KFC': u"YRD_KF", 
    u'Sonic Drive-In': u"SRI", u'Popeyes Louisiana Kitchen': u"POP",
    u'In-N-Out Burger': u"In-N-Out Burger",  u'Little Caesars Pizza': "Little Caesars",
    u"Domino's Pizza": u'DOM', u"Chili's Grill & Bar": "Brinker International",
    u'Olive Garden Italian Restaurant': u'DAR', u'Jack In The Box': "JACK",
    u"Jimmy John's": u"Jimmy John's", u'Red Lobster': "REDL",
    u"Moe's Southwest Grill": u"Moe's Southwest Grill", u'Texas Roadhouse': u"Texas Roadhouse",
    u'Golden Corral': u"Golden Corral", u"Dunkin' Donuts": u"DUN",
    u"Denny's": u"DENN", u'QDOBA Mexican Eats': "QDO", u'Wingstop': "Roark Capital Group",
    u'El Pollo Loco': u"El Pollo Loco", u"Raising Cane's Chicken Fingers": u"Raising Cane's",
    u'The Cheesecake Factory': u"CHE", u"Zaxby's Chicken Fingers & Buffalo Wings": u"Zaxby's",
    u'Whataburger': 'WHT', u'Five Guys': u"Five Guys", u'Del Taco': "Del Taco",
    u'Waffle House': u"Waffle House, Inc.", u'MOD Pizza': u"MOD Pizza", u'Potbelly Sandwich Shop': u"POT",
    u"Bojangles' Famous Chicken 'n Biscuits": u"BOJA", u'Baskin-Robbins': u"BR",
    u'Bonefish Grill': u"OUTB_BON", u'California Pizza Kitchen': u"CPKI", u"Captain D's": u"CAP",
    u"Carl's Jr.": u"CKE Restaurants", u"Carrabba's Italian Grill": u"OUTB_CAR",
    u"Casey's General Store": u"Casey's General Store", u'Checkers': u"CHEC", u"Church's Chicken": u"CHU",
    u'Corner Bakery Cafe': u"CBC", u'Dairy Queen': "DAI", u"Eddie V's Prime Seafood": "DAR",
    u"Famous Dave's Bar-B-Que": u"FAMD", u"Fazoli's": u"FAZ",
    u'Firehouse Subs': u"FIRE", u"Friendly's": u"FRIE", u"Hardee's": u"CKE Restaurants",  u'Jamba Juice': u"JAMB",
    u"Jason's Deli": u"Jason's Deli", u"Jersey Mike's Subs": u"Jersey Mike's Subs",
    u"Jimmy John's": u"Jimmy John's", u'Krispy Kreme Doughnuts': u"KRI", u'Kroger': u"KRO",
    u"Logan's Roadhouse": u"LORH", u"Long John Silver's": u"LONG", u'LongHorn Steakhouse': u"DAR",
    u'Noodles and Company': u"NOO", u'On The Border Mexican Grill & Cantina': u"On The Border",
    u'Outback Steakhouse': u"OUTB", u"P.F. Chang's": u"PFC", u'Panda Express': u"Panda Express",
    u"Papa Murphy's Take 'N' Bake Pizza": u"Papa Murphy's", u'Pei Wei': u"PFC",u'Pollo Tropical': u"FIES_TROP",
    u'QuickChek': u"QuickChek", u'QuikTrip': u"QuikTrip", u"Rally's": u"CHEC", 
    u'Red Robin Gourmet Burgers': u"REDR", u'Regal Cinemas': u"Regal Entertainment Group",
    u"Romano's Macaroni Grill": u"Ignite Restaurant Group", u'Shake Shack': u"Shake Shack",
    u'Smashburger': u"Smashburger", u'Steak N Shake': u"Steak 'n Shake", 
    u"Ted's Montana Grill": u"Ted's Montana Grill", u'Zo\xebs Kitchen': u"Zo\xebs Kitchen",
    u"White Castle": u"White Castle", u"Wawa": u"WAWA", u"Culver's": u"CULV", u"Krystal": u"KRY",
    u"Taco Cabana": u"FIES_TACO", u"Boston Market": u"BOSM", u"Ruby Tuesday": u"RUBY", 
    u"Baja Fresh": u"Fresh Enterprises, LLC", u"Yard House": u"DAR", u"Safeway": u"SAFE", u"vons": u"SAFE",
    u"Cheddar's Scratch Kitchen":u"Cheddar's", u"Portillo's Hot Dogs": u"PORT",
    u"BJ's Restaurant & Brewhouse": u"BJS Restaurant & Brewhouse", u"Trader Joe's": u"Trader Joe's",
    u"Whole Foods Market": u"Whole Foods", u"Pret A Manger": u"Pret a Manger UK", 
    u"Sheetz": u"Shee", "Joe's Crab Shack": "JOES", u"Schlotzsky's": u"Focus Brands",
    u"Cracker Barrel Old Country Store": u"CRA", u"Houlihan's": u"HOU", "marco's pizza": u"MARC",
    u"which wich superior sandwiches": u"WHIC", u"Papa John's Pizza": u"PAPA", u'Wienerschnitzel': u"GALR",
    u"einstein bros. bagels": u"EIN", u"tropical smoothie cafe": u"TROP", "meijer": u"MEIJ", 
    u"caribou coffee": u"CARI"}
           
brand_prefix_dict = {k.lower():v + "_IWP" for (k,v) in brand_prefix_dict.items()}

brand_variation_dict = {u'Chick-fil-A': [u'Chick-fil-A Avalon', u'Chick-fil-A Bangor',
                                         u'Chick-fil-A Eastern and Ione', u'Chick-fil-A Mowry Ave',
                                         u'Chick-fil-A North Decatur', u'Chick-fil-A Pearland Parkway',
                                         u'Chick-fil-A Tampa Stadium', u'Chick-fil-A Westnedge',
                                         u'Chick-fil-A Worcester', u'Chick-fil-A at Glendora Commons',
                                         u'Chick-fil-A at Melrose Park', u"chick fil a",
                                         u"chick-fil-a at northeast tower center"],
            u'California Pizza Kitchen': [u'California Pizza Kitchen at Hill Center',
                                          u'California Pizza Kitchen at Willow Bend',],
            u'Jamba Juice': [u'Jamba Juice Amerige Center', u'Jamba Juice Larchmont Ave',],
            u'Shake Shack': [u'Shake Shack - Coral Gables', u'Shake Shack - Kierland Commons',
                  u'Shake Shack - MGM NY NY', u'Shake Shack - West Loop', u'Shake Shack - Winter Park',],
            u"Bojangles' Famous Chicken 'n Biscuits": [u"Bojangles' Coliseum"],
            u'Dairy Queen': [u'Dairy Queen Grill & Chill - Cary', "DQ Grill & Chill", "dairy queen (treat only)"],
            u'Krispy Kreme Doughnuts':[ u'Krispy Kreme Evergreen Park'],
            u'SUBWAY\xaeRestaurants': [u'Subway', u"subway resturant inside chi", u"subway® restaurants",
                                       u"subway south commercial"],
            u'Steak N Shake': ["Steak 'n Shake"], u"Whole Foods Market": [u"whole foods"],
            u"Chipotle Mexican Grill": ["Chipotle", u"chipotle", "chipotle mexican grill ."],
            u"Souplantation": ["Garden Fresh"], u'Kroger': [u"kroger marketplace"],
            u"Raising Cane's Chicken Fingers": ["Raising Cane's"],
            u'Applebee\u2019s': [u"applebee’s grill + bar", u"applebee's", u"applebee’s grill + bar",
                                 u"applebee\u2019s grill + bar"],
            u"AMC":["amc hoffman center 22", "amc classic apple blossom 12"],
            u"Chili's Grill & Bar" :[u"chili's grill & bar hagerstown"],
            u'Pizza Hut': [u"pizza hut express"], u"Wawa":["wawa 4455 sr 64 east"],
            u"Papa John's Pizza":[u'Papa Johns', u"papa john's pizza", u"papa john\u2019s pizza"],
            u'Starbucks':[u"starbucks reserve"], u'QDOBA Mexican Eats': [u"qdoba mexican grill"],
            u"Carl's Jr.": [u"carl's jr. / green burrito"], u'Golden Corral': [u"golden corral elyria"],
            }

brand_normalization = {}
for k,values_list in brand_variation_dict.items():
    for value in values_list:
        brand_normalization[value.lower()] = k.lower()

# Query SMG Database for Historical IWP.com Reports
- Current: read all past records in from .csv 
- Future: think we just need legacy_stop_date from the historical data. Need to confirm no duplicates could possibly result

In [5]:
input_file = "iwaspoisoned_scraped*.xlsx"

list_of_files = glob.glob(data_path + input_file) 
if len(list_of_files)>0:
    latest_file = max(list_of_files, key=os.path.getctime)
    print latest_file
    df_ht_iwp_legacy_raw = pd.read_excel(latest_file, 'IWP_Raw', converters = my_formats)
    df_ht_iwp_legacy_raw = df_ht_iwp_legacy_raw[my_cols]
    
print len(df_ht_iwp_legacy_raw.index)

/Users/loubellaire/Documents/_SMG/Epic_Planning/DataScience/Text Processing/Alerts/food_illness/IWP_tableformat/iwaspoisoned_scraped20180919_1805.xlsx
7023


# IWP.com Webspider Code
- Most recent 30-days of reporting is available via IWP.com website 
- reports which include consumer complaints as well as health department reports (identified as Report_Type = "Temporary Closure")
- This function scrapes IWP.com website to fill in gaps between today and last database update
- A UID is created format is Date_IWP_hash(location)
- We use usaddress.py (open source library) to parse address information. Implementation is best available however it is less than perfect.

In [6]:
'''
Simple Test Code:
       r = requests.get('https://iwaspoisoned.com/?page=1')
       soup_0 = BeautifulSoup(r.text, 'html.parser') # "lxml")

webspider termination test

now = datetime.now()
df_ht_iwp_legacy['delta'] = now - df_ht_iwp_legacy["date_time"].astype(datetime)
legacy_stop_date = df_ht_iwp_legacy[df_ht_iwp_legacy['delta'] == min(df_ht_iwp_legacy['delta'])]["date_time"][0]

ambiguous_rec_date = datetime.strptime(attributes_dict["date_time"], '%b %d %H:%M%p')
candidates_rec_dto = [ambiguous_rec_date.replace(year=datetime.now().year),\
                      ambiguous_rec_date.replace(year=(datetime.now()-relativedelta(years=1)).year)]
current_rec_dto = nearest(candidates_rec_dto, now)

print (current_rec_dto - stop_date)

if current_rec_dto >= stop_date:
    continue
else:
    break
'''

stamper = lambda s: unicode(s.strftime('%Y%m%d%H%M%S') + "IWP")
hasher = lambda s: base64.urlsafe_b64encode(hashlib.sha256(s.encode('utf-8')).hexdigest()[0:26])
date_coverage = df_ht_iwp_legacy_raw["date_time"].map(lambda x: x.strftime('%Y-%m-%d')).unique()

now = datetime.now()
df_ht_iwp_legacy_raw['delta'] = now - df_ht_iwp_legacy_raw["date_time"].astype(datetime)
legacy_stop_date = df_ht_iwp_legacy_raw[df_ht_iwp_legacy_raw['delta'] ==\
                                        min(df_ht_iwp_legacy_raw['delta'])]["date_time"].tolist()[0]
print legacy_stop_date

data_list = [] 
count = 0
for i in xrange(1,366):
    time.sleep(7)
    sit_reports = []
    try:
        r = requests.get('https://iwaspoisoned.com/?page=' + str(i))
        soup = BeautifulSoup(r.text, 'html.parser') # "lxml")
        #sit_reports = soup.find_all('div', class_="post-box")
        sit_reports = soup.find_all('div', class_="div-report-box-small")
    
    except:
        time.sleep(80)
        count +=1
        print "page: {0}, error count: {1}".format(i, count)
        if count > 32:
            break

    for report in sit_reports:
        attributes_dict = {}
        if u'USA' in report.find('div', class_ = "col-md-6 report-first-box pull-left").find('a').get_text():
            ambiguous_rec_date = report.find('p', class_ = "report-date").get_text()
            ambiguous_rec_date = datetime.strptime(ambiguous_rec_date, '%b %d %H:%M%p')
            candidate_rec_dtos = [ambiguous_rec_date.replace(year=datetime.now().year),\
                                  ambiguous_rec_date.replace(year=(datetime.now()-relativedelta(years=1)).year)]
            attributes_dict['date_time'] = nearest(candidate_rec_dtos, now)
            if attributes_dict['date_time'] < legacy_stop_date:
                print "Date exceeded", attributes_dict['date_time']
                break
            s = report.find('h3', class_= "report-box-h").get_text()
            attributes_dict['Location'] = s
            attributes_dict['IWP_brand'] = s.split(",")[0]
            attributes_dict['source'] = "iwaspoisoned.com"
            
            if report.find('p', class_= "report-tag") is not None:
                t = report.find('p', class_= "report-tag").get_text()
                attributes_dict['Symptoms'] = t.split(":")[1].strip()
                attributes_dict['Report Type'] = "Food Poisoning"
                attributes_dict["StreetNamePostDirectional"] = None
                attributes_dict["Details"] = report.find("div", class_="col-md-6 report-second-box pull-left").find("p").get_text()
            else:
                continue
                
            my_odict =  OrderedDict({})
            try:
                my_odict = usaddress.tag(','.join(s.split(',')[1:-1]))[0]
            except usaddress.RepeatedLabelError as e :
                LandmarkName_list = [item[0] for item in e.parsed_string if item[1] == 'LandmarkName']
                if len(LandmarkName_list) > 1:
                    my_odict["LandmarkName"] = ' '.join(LandmarkName_list)
                StateName_list = [item[0] for item in e.parsed_string if item[1] == 'StateName']
                if len(StateName_list) > 1:
                    my_odict["StateName"] = min(StateName_list, key=len)
                PlaceName_list = [item[0] for item in e.parsed_string if item[1] == 'PlaceName_list']
                if len(PlaceName_list) > 1:
                    my_odict['PlaceName'] = PlaceName_list[0]
            except:
                continue

            v = None
            for k,v in my_odict.items():
                attributes_dict[k] = v
            data_list.append(copy.deepcopy(attributes_dict))
            attributes_dict.clear()
            my_odict.clear()
    else:                                 # see: http://psung.blogspot.com.au/2007/12/for-else-in-python.html
        continue
    break

data = pd.DataFrame(data_list)
drop_list = list(set(data.columns.values).intersection(set(['CornerOf', 'LandmarkName',\
                                                            'Recipient', u'Closure Date', 'NotAddress'])))
data = data.drop(drop_list, axis=1)
cols_withnulls = ['Diagnosis', 'Doctor Visit', 'BuildingName', 'CountryName']
data[cols_withnulls] = pd.DataFrame([[None, None, None, None]], index=data.index)  

data['IWP_brand'] = data['IWP_brand'].apply(stripspaces).str.lower()
data['IWP_brand'] = data['IWP_brand'].replace(brand_normalization)

data['Brand_prefix'] = data['IWP_brand'].str.lower().map(brand_prefix_dict)
data['Brand_prefix'] = data['Brand_prefix'].where((pd.notnull(data['Brand_prefix'])), None)

#df['source'] = "iwaspoisoned.com"
#df['CountryName'] = 'USA'
#data['date_time'] = pd.to_datetime(data['date_time'], format='%b %d %H:%M%p %Y')

data['stamped'] = data['date_time'].apply(stamper)
data['hash'] = data['Location'].apply(hasher)
data['uid'] = data[['stamped','hash']].apply(lambda x: ''.join(x.dropna().astype(unicode)),axis=1)

missing_cols = list(set(my_cols) - set(data.columns))
if len(missing_cols)>0:
    print "bingo"
    data = data.reindex(columns=my_cols, fill_value=np.nan)
    
df = data[~data["Report Type"].str.contains("Temporary Closure")]

df = df[my_cols]

2018-09-19 12:57:00
page: 73, error count: 1
Date exceeded 2018-09-19 09:42:00


# Correct Geo's and enrich IWP.com
- Future to look at google api to correct usaddress deficiencies.
- Add state FIPS code to enable map displays

In [7]:
if df_ht_iwp_legacy_raw.index.name == 'uid':
    df_ht_iwp_legacy_raw.reset_index(inplace = True)
if df.index.name =='uid':
    df.reset_index(inplace = True)

df2 = df[~df['uid'].isin(df_ht_iwp_legacy_raw['uid'])].dropna(how = 'all')
df_ht_iwp_legacy_update = pd.concat([df_ht_iwp_legacy_raw, df2])
df_ht_iwp_legacy_update = df_ht_iwp_legacy_update[my_cols]

df_correct = pd.merge(df_ht_iwp_legacy_update, df_fips[['State','USPS']],\
                                   left_on='StateName', right_on= 'State', how='left')
df_correct['USPS2'] = df_correct[['StateName', "USPS"]].apply(merged_states, axis = 1)
df_correct.drop(['StateName', 'State', "USPS"], axis=1, inplace = True)
df_correct.rename(columns ={'USPS2':'StateName'}, inplace = True)
df_final = pd.merge(df_correct,df_fips[['USPS','FIPS']], left_on='StateName', right_on='USPS', how='left')
df_final.drop(["USPS"], axis=1, inplace = True)

print len(df_final.index)

7467


# Update SMG Historical Database of scraped IWP.com Reports
- Current: write all records out to .csv 
- Future: udpate Hive table with new information

In [8]:
df_final['IWP_brand'] = df_final['IWP_brand'].apply(stripspaces).str.lower()
df_final['IWP_brand'] = df_final['IWP_brand'].replace(brand_normalization)

df_final['Brand_prefix'] = df_final['IWP_brand'].str.lower().map(brand_prefix_dict)
df_final['Brand_prefix'] = df_final['Brand_prefix'].where((pd.notnull(df_final['Brand_prefix'])), None)
df_final.sort_values(by = "date_time", ascending = True, inplace = True)

In [9]:
data_path = "/Users/loubellaire/Documents/_SMG/Epic_Planning/DataScience/Text Processing/Alerts/food_illness/IWP_tableformat/"
output_file = "iwaspoisoned_scraped{date:%Y%m%d_%H%M}.xlsx".format(date=datetime.now())

writer = pd.ExcelWriter(data_path + output_file, engine='xlsxwriter', options={'encoding':'utf-8'})
df_final.to_excel(writer, sheet_name='IWP_Raw')
writer.save()

print datetime.now()
print "Number of new Entries: {} ".format(len(df.index))
print df_final['date_time'][0]

2018-10-03 10:37:19.548990
Number of new Entries: 444 
2018-04-25 09:23:00


In [10]:
print data_path + output_file
print df_final['date_time'][0]

/Users/loubellaire/Documents/_SMG/Epic_Planning/DataScience/Text Processing/Alerts/food_illness/IWP_tableformat/iwaspoisoned_scraped20181003_1037.xlsx
2018-04-25 09:23:00


In [ ]:
# INSERT INTO DATABASE: Prepare data to be added to the Hive database

In [ ]:
# Notes about HDFS
#   It is a distributed file system, so you cannot use standard unix commands like ls, cd. Instead
# there is a group of commands with a prefix hdfs to help navigate and view the directory structure.
# For example, to navigate to a subfolder you would cmd = 'hdfs cd ' + hdfs_path or 'hdfs ls ' + hdfs path.  
# To recursively execute a chmod or mkdir rm, you may need to add hdfs -r to the command.

# Hive notes: ticks should wrap each object, not the fully-qualified name
# for example, c.execute("DROP TABLE `default`.`bk_comment_word_freq_with_stops`")

hdfs_path = "/user/jupyter/LFHC_alerts/foodborne_illness/"

cmd='hdfs dfs -mkdir ' + hdfs_path
result=subprocess.check_output(cmd, shell=True)
print result

cmd='hdfs dfs -chmod 777 ' + hdfs_path
result=subprocess.check_output(cmd, shell=True)
print result

# View directory contents
cmd='hdfs dfs -ls ' + hdfs_path
result = subprocess.check_output(cmd, shell=True)
print;print result

with pyhs2.connect(host=hiveHost, port=hivePort, authMechanism="PLAIN", user='elemmon') as cn:
    with cn.cursor() as c:
        c.execute(""" CREATE EXTERNAL TABLE `{0}`(
                `uid` string,
                `source` string,
                `IWP_brand` string,
                `Brand_prefix` string,
                `date_time` TIMESTAMP,
                `Report Type` string,
                `Symptoms` string,
                `Diagnosis` string,
                `Doctor Visit` string,
                `Details` string,
                `AddressNumber` string,
                `BuildingName` string,
                `CountryName` string,
                `Location` string,
                `PlaceName` string,
                `StreetName` string,
                `StreetNamePostDirectional` string,
                `StreetNamePostType` string,
                `StreetNamePreDirectional` string,
                `ZipCode` int,
                `FIPS` int,) 
             ROW FORMAT SERDE
             'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
             WITH SERDEPROPERTIES (
             'field.delim'=',',
             'serialization.format'=',') 
             STORED AS INPUTFORMAT 
             'org.apache.hadoop.mapred.TextInputFormat'
             OUTPUTFORMAT
             'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
             LOCATION
             'hdfs://smgdev""".format(df_final) + hdfs_path + "'")

In [ ]:
print brand_prefix_dict[u"chili's grill & bar"]
print "olive garden italian restaurant", "buffalo wild wings", "applebee’s", "papa john's pizza", "ihop", "pizza hut"

# Code to address Historical Data deficiencies
```
df_ht_iwp_legacy_raw.rename({'brand': 'IWP_brand'}, axis='columns', inplace = True)
df_ht_iwp_legacy_raw['IWP_brand'] = df_ht_iwp_legacy_raw['IWP_brand'].apply(stripspaces).str.lower()

df_ht_iwp_legacy_raw['IWP_brand'] = df_ht_iwp_legacy_raw['IWP_brand'].replace(brand_normalization)

df_ht_iwp_legacy_raw['Brand_prefix'] = df_ht_iwp_legacy_raw['IWP_brand'].map(brand_prefix_dict)
df_ht_iwp_legacy_raw['Brand_prefix'] = df_ht_iwp_legacy_raw['Brand_prefix'].\
                                           where((pd.notnull(df_ht_iwp_legacy_raw['Brand_prefix'])), None) 
df_ht_iwp_legacy_raw['CountryName'] = 'USA'
df_ht_iwp_legacy_raw.drop([u'relevant', u'products'], axis=1, inplace=True)

df_ht_iwp_legacy_raw =df_ht_iwp_legacy_raw[my_cols]

mypath = "/Users/loubellaire/Documents/_SMG/Epic_Planning/DataScience/Text Processing/Alerts/food_illness/IWP_tableformat/"
output_file = "iwaspoisoned_scraped{date:%Y%m%d_%H%M}.xlsx".format(date=datetime.now())

writer = pd.ExcelWriter(mypath + output_file, engine='xlsxwriter', options={'encoding':'utf-8'})
df_ht_iwp_legacy_raw.to_excel(writer, sheet_name='SourceData')
writer.save()
```